In [1]:
!pip install numpy pandas scikit-learn nltk keras tensorflow flask


  Using cached keras-3.7.0-py3-none-any.whl.metadata (5.8 kB)
Using cached keras-3.7.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install --upgrade tensorflow
!pip install --upgrade keras



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, ClassifierMixin

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dense, LSTM, Input




nltk.download('stopwords')
from nltk.corpus import stopwords



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/luisdavid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
!pip install kaggle



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pwd

/Users/luisdavid/Desktop/9A/Mario/U3


In [6]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
!kaggle datasets download -d kritanjalijain/amazon-reviews

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/__init__.py", line 3, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/api/__init__.py", line 6, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 258, in <module>
    class KaggleApi(KaggleApi):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 289, in KaggleApi
    os.makedirs(config_dir)
  File "<frozen os>", line 225, in makedirs
OSError: [Errno 30] Read-only file system: '/content'


In [121]:
!unzip amazon-reviews.zip -d amazondata


Archive:  amazon-reviews.zip
replace amazondata/amazon_review_polarity_csv.tgz? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


Este código realiza la preparación y preprocesamiento de datos para entrenar modelos de Machine Learning y Deep Learning a partir de datos de texto (en este caso, reseñas de Amazon)

In [108]:
# Parte 1: Preparación y Preprocesamiento

import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Función para cargar y formatear datos
def load_data(file_path):
    # Cargar datos sin encabezados
    data = pd.read_csv(file_path, header=None)
    # Nombrar columnas basándonos en el formato proporcionado
    data.columns = ["label", "title", "review_text"]
    return data

# Función para limpiar texto
def clean_text(text):
    # Quitar caracteres especiales, puntuaciones y números
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'\s+', ' ', text).strip()  # Eliminar espacios adicionales
    return text

# Preprocesamiento general
def preprocess_data(data):
    # Limpiar la columna de texto
    data["cleaned_review_text"] = data["review_text"].apply(clean_text)
    # Convertir etiquetas a 0 (negativo) y 1 (positivo)
    data["label"] = data["label"].apply(lambda x: 0 if x == 1 else 1)
    return data

# Vectorización TFIDF
def prepare_tfidf_vectorizer(X_train, X_test):
    vectorizer = TfidfVectorizer(max_features=50000, stop_words='english')
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    return vectorizer, X_train_tfidf, X_test_tfidf

# Preparación para DNN
def prepare_dnn_data(X_train, X_test, max_vocab_size=50000, max_sequence_length=100):
    tokenizer = Tokenizer(num_words=max_vocab_size)
    tokenizer.fit_on_texts(X_train)
    X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_sequence_length)
    X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_sequence_length)
    return tokenizer, X_train_seq, X_test_seq

# Rutas de los datos
train_path = "amazondata/train.csv"
test_path = "amazondata/test.csv"

# Cargar y preprocesar datos
train_data = load_data(train_path)
test_data = load_data(test_path)

train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)

# División en características y etiquetas
X_train, y_train = train_data["cleaned_review_text"], train_data["label"]
X_test, y_test = test_data["cleaned_review_text"], test_data["label"]

# Preparar datos para TFIDF
vectorizer, X_train_tfidf, X_test_tfidf = prepare_tfidf_vectorizer(X_train, X_test)

# Preparar datos para DNN
tokenizer, X_train_seq, X_test_seq = prepare_dnn_data(X_train, X_test)

print("Datos listos para los modelos:")
print(f"TFIDF - X_train: {X_train_tfidf.shape}, X_test: {X_test_tfidf.shape}")
print(f"DNN - X_train_seq: {X_train_seq.shape}, X_test_seq: {X_test_seq.shape}")




Datos listos para los modelos:
TFIDF - X_train: (3600000, 50000), X_test: (399997, 50000)
DNN - X_train_seq: (3600000, 100), X_test_seq: (399997, 100)


2. Red Neuronal Profunda (DNN)
Se instancia la clase DNNClassifier especificando:

vocab_size: Tamaño del vocabulario.

sequence_length: Longitud máxima de las secuencias de entrada.

embedding_dim: Dimensión de los vectores de embeddings.

El modelo se entrena en los datos de entrenamiento (X_train_seq y y_train) usando:

3 épocas (epochs).

Tamaño de lote de 64 (batch_size).

Validación con un conjunto de prueba (X_test_seq, y_test).

Se evalúa el modelo generando un informe de clasificación (classification_report), que incluye métricas detalladas para cada clase.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report
import numpy as np

class DNNClassifier:
    def __init__(self, vocab_size=50000, sequence_length=100, embedding_dim=128):
        self.model = Sequential([
            Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_length),
            LSTM(128, return_sequences=True),
            Dropout(0.5),
            LSTM(64),
            Dense(32, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    def train(self, X_train, y_train, epochs=5, batch_size=32, validation_data=None):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    def predict(self, X_test):
        predictions = self.model.predict(X_test)
        return (predictions > 0.5).astype(int).flatten()

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        report = classification_report(y_test, y_pred, target_names=["Negative", "Positive"])
        return report

# Entrenamiento y evaluación del clasificador
dnn_classifier = DNNClassifier(vocab_size=50000, sequence_length=100)
dnn_classifier.train(X_train_seq, y_train, epochs=3, batch_size=64, validation_data=(X_test_seq, y_test))

# Informe de clasificación
dnn_report = dnn_classifier.evaluate(X_test_seq, y_test)
print("DNN Classification Report:")
print(dnn_report)


Epoch 1/3
56250/56250 ━━━━━━━━━━━━━━━━━━━━ 7348s 131ms/step - accuracy: 0.9051 - loss: 0.2323 - val_accuracy: 0.9348 - val_loss: 0.1700
Epoch 2/3
56250/56250 ━━━━━━━━━━━━━━━━━━━━ 7234s 129ms/step - accuracy: 0.9428 - loss: 0.1522 - val_accuracy: 0.9381 - val_loss: 0.1624
Epoch 3/3
56250/56250 ━━━━━━━━━━━━━━━━━━━━ 7410s 132ms/step - accuracy: 0.9511 - loss: 0.1324 - val_accuracy: 0.9377 - val_loss: 0.1634
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 294s 23ms/step
DNN Classification Report:
              precision    recall  f1-score   support

    Negative       0.93      0.94      0.94    200000
    Positive       0.94      0.93      0.94    200000

    accuracy                           0.94    400000
   macro avg       0.94      0.94      0.94    400000
weighted avg       0.94      0.94      0.94    400000

